In [9]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr, col, lit
spark = SparkSession.builder.appName("Jupyter").getOrCreate()

df = spark.read.option("header", "true") \
            .csv("/home/iceberg/data/events.csv") \
            .withColumn("event_date", expr("DATE_TRUNC('day', event_time)"))

df.collect()

[Row(user_id='1037710827', device_id='532630305', referrer=None, host='www.zachwilson.tech', url='/', event_time='2021-03-08 17:27:24.241000', event_date=datetime.datetime(2021, 3, 8, 0, 0)),
 Row(user_id='925588856', device_id='532630305', referrer=None, host='www.eczachly.com', url='/', event_time='2021-05-10 11:26:21.247000', event_date=datetime.datetime(2021, 5, 10, 0, 0)),
 Row(user_id='-1180485268', device_id='532630305', referrer=None, host='admin.zachwilson.tech', url='/', event_time='2021-02-17 16:19:30.738000', event_date=datetime.datetime(2021, 2, 17, 0, 0)),
 Row(user_id='-1044833855', device_id='532630305', referrer=None, host='www.zachwilson.tech', url='/', event_time='2021-09-24 15:53:14.466000', event_date=datetime.datetime(2021, 9, 24, 0, 0)),
 Row(user_id='747494706', device_id='532630305', referrer=None, host='www.zachwilson.tech', url='/', event_time='2021-09-26 16:03:17.535000', event_date=datetime.datetime(2021, 9, 26, 0, 0)),
 Row(user_id='747494706', device_id='

In [13]:
sorted = df.repartition(10, col("event_date")) \
        .sortWithinPartitions(col("event_date"), col("host")) \
        .withColumn("event_time", col("event_time").cast("timestamp"))

sortedTwo = df.repartition(10, col("event_date")) \
        .sort(col("event_date"), col("host")) \
        .withColumn("event_time", col("event_time").cast("timestamp"))


In [14]:
%%sql

CREATE DATABASE IF NOT EXISTS bootcamp

++
||
++
++

In [15]:
%%sql

DROP TABLE IF EXISTS bootcamp.events

++
||
++
++

In [28]:
%%sql

CREATE TABLE IF NOT EXISTS bootcamp.events (
    url STRING,
    referrer STRING,
    browser_family STRING,
    os_family STRING,
    device_family STRING,
    host STRING,
    event_time TIMESTAMP,
    event_date DATE
)
USING iceberg
PARTITIONED BY (event_date);




++
||
++
++

In [24]:
%%sql

CREATE TABLE IF NOT EXISTS bootcamp.events_sorted (
    url STRING,
    referrer STRING,
    browser_family STRING,
    os_family STRING,
    device_family STRING,
    host STRING,
    event_time TIMESTAMP,
    event_date DATE
)
USING iceberg
PARTITIONED BY (event_date);

++
||
++
++

In [26]:
%%sql
CREATE TABLE IF NOT EXISTS bootcamp.events_unsorted (
    url STRING,
    referrer STRING,
    browser_family STRING,
    os_family STRING,
    device_family STRING,
    host STRING,
    event_time TIMESTAMP,
    event_date DATE
)
USING iceberg
PARTITIONED BY (event_date);



++
||
++
++

In [29]:

start_df = df.repartition(4, col("event_date")).withColumn("event_time", col("event_time").cast("timestamp")) \
    
first_sort_df = start_df.sortWithinPartitions(col("event_date"), col("host"))

start_df.write.mode("overwrite").saveAsTable("bootcamp.events_unsorted")
first_sort_df.write.mode("overwrite").saveAsTable("bootcamp.events_sorted")

In [30]:
%%sql

SELECT SUM(file_size_in_bytes) as size, COUNT(1) as num_files, 'sorted' 
FROM demo.bootcamp.events_sorted.files

UNION ALL
SELECT SUM(file_size_in_bytes) as size, COUNT(1) as num_files, 'unsorted' 
FROM demo.bootcamp.events_unsorted.files





size,num_files,sorted
4932864,4,sorted
5015266,4,unsorted


In [31]:
%%sql
SELECT SUM(file_size_in_bytes) as size, COUNT(1) as num_files FROM demo.bootcamp.events.files;

size,num_files
None,0


In [33]:
%%sql 
SELECT * FROM bootcamp.matches_bucketed.files

content,file_path,file_format,spec_id,partition,record_count,file_size_in_bytes,column_sizes,value_counts,null_value_counts,nan_value_counts,lower_bounds,upper_bounds,key_metadata,split_offsets,equality_ids,sort_order_id,readable_metrics
0,s3://warehouse/bootcamp/matches_bucketed/data/completion_date=2015-10-27T00%3A00Z/match_id_bucket=1/00000-13-6ffc0dc7-82e0-4206-9d0c-84aaa989ffa1-00001.parquet,PARQUET,0,"Row(completion_date=datetime.datetime(2015, 10, 27, 0, 0), match_id_bucket=1)",1,1721,"{1: 88, 2: 50, 3: 88, 4: 57}","{1: 1, 2: 1, 3: 1, 4: 1}","{1: 0, 2: 0, 3: 0, 4: 0}",{},"{1: bytearray(b'ddb476eb-3323-4c'), 2: bytearray(b'\x01'), 3: bytearray(b'f27a65eb-2d11-49'), 4: bytearray(b'\x00\xa0L\xc2\n#\x05\x00')}","{1: bytearray(b'ddb476eb-3323-4d'), 2: bytearray(b'\x01'), 3: bytearray(b'f27a65eb-2d11-4:'), 4: bytearray(b'\x00\xa0L\xc2\n#\x05\x00')}",None,[4],None,0,"Row(completion_date=Row(column_size=57, value_count=1, null_value_count=0, nan_value_count=None, lower_bound=datetime.datetime(2015, 10, 27, 0, 0), upper_bound=datetime.datetime(2015, 10, 27, 0, 0)), is_team_game=Row(column_size=50, value_count=1, null_value_count=0, nan_value_count=None, lower_bound=True, upper_bound=True), match_id=Row(column_size=88, value_count=1, null_value_count=0, nan_value_count=None, lower_bound='ddb476eb-3323-4c', upper_bound='ddb476eb-3323-4d'), playlist_id=Row(column_size=88, value_count=1, null_value_count=0, nan_value_count=None, lower_bound='f27a65eb-2d11-49', upper_bound='f27a65eb-2d11-4:'))"
0,s3://warehouse/bootcamp/matches_bucketed/data/completion_date=2015-10-27T00%3A00Z/match_id_bucket=2/00000-13-6ffc0dc7-82e0-4206-9d0c-84aaa989ffa1-00002.parquet,PARQUET,0,"Row(completion_date=datetime.datetime(2015, 10, 27, 0, 0), match_id_bucket=2)",2,1675,"{1: 115, 2: 50, 3: 115, 4: 98}","{1: 2, 2: 2, 3: 2, 4: 2}","{1: 0, 2: 0, 3: 0, 4: 0}",{},"{1: bytearray(b'21808f7e-5169-44'), 2: bytearray(b'\x01'), 3: bytearray(b'892189e9-d712-4b'), 4: bytearray(b'\x00\xa0L\xc2\n#\x05\x00')}","{1: bytearray(b'a4142671-41b4-4:'), 2: bytearray(b'\x01'), 3: bytearray(b'f27a65eb-2d11-4:'), 4: bytearray(b'\x00\xa0L\xc2\n#\x05\x00')}",None,[4],None,0,"Row(completion_date=Row(column_size=98, value_count=2, null_value_count=0, nan_value_count=None, lower_bound=datetime.datetime(2015, 10, 27, 0, 0), upper_bound=datetime.datetime(2015, 10, 27, 0, 0)), is_team_game=Row(column_size=50, value_count=2, null_value_count=0, nan_value_count=None, lower_bound=True, upper_bound=True), match_id=Row(column_size=115, value_count=2, null_value_count=0, nan_value_count=None, lower_bound='21808f7e-5169-44', upper_bound='a4142671-41b4-4:'), playlist_id=Row(column_size=115, value_count=2, null_value_count=0, nan_value_count=None, lower_bound='892189e9-d712-4b', upper_bound='f27a65eb-2d11-4:'))"
0,s3://warehouse/bootcamp/matches_bucketed/data/completion_date=2015-10-27T00%3A00Z/match_id_bucket=4/00000-13-6ffc0dc7-82e0-4206-9d0c-84aaa989ffa1-00003.parquet,PARQUET,0,"Row(completion_date=datetime.datetime(2015, 10, 27, 0, 0), match_id_bucket=4)",2,1775,"{1: 115, 2: 50, 3: 130, 4: 98}","{1: 2, 2: 2, 3: 2, 4: 2}","{1: 0, 2: 0, 3: 0, 4: 0}",{},"{1: bytearray(b'4f55c159-6e5c-42'), 2: bytearray(b'\x01'), 3: bytearray(b'f27a65eb-2d11-49'), 4: bytearray(b'\x00\xa0L\xc2\n#\x05\x00')}","{1: bytearray(b'c4bd1739-3327-4d'), 2: bytearray(b'\x01'), 3: bytearray(b'f27a65eb-2d11-4:'), 4: bytearray(b'\x00\xa0L\xc2\n#\x05\x00')}",None,[4],None,0,"Row(completion_date=Row(column_size=98, value_count=2, null_value_count=0, nan_value_count=None, lower_bound=datetime.datetime(2015, 10, 27, 0, 0), upper_bound=datetime.datetime(2015, 10, 27, 0, 0)), is_team_game=Row(column_size=50, value_count=2, null_value_count=0, nan_value_count=None, lower_bound=True, upper_bound=True), match_id=Row(column_size=115, value_count=2, null_value_count=0, nan_value_count=None, lower_bound='4f55c159-6e5c-42', upper_bound='c4bd1739-3327-4d'), playlist_id=Row(column_size=130, value_count=2, null_value_count=0, nan_value_count=N